In [94]:
import fml_manager
import json, time, requests
import os

manager = fml_manager.FMLManager()

In [39]:
response = manager.load_data("./data/breast_b.csv", "fate_flow_test_breast", "breast", 1, 1, 10, 1, api_version="1.4")
output = json.loads(response.content)
print(output)
guest_job_id = output["jobId"]
guest_query_condition = {
    "job_id":guest_job_id
}

for i in range(500):
    time.sleep(1)
    guest_status = manager.query_job(guest_query_condition).json()["data"][0]["f_status"]

    print("{}".format(guest_status))
    
    if guest_status == "failed":
        print("Failed")
        raise Exception("Failed to run the jobs")
    if guest_status == "success":
        print("Success")       
        break

{'data': {'board_url': 'http://fateboard:8080/index.html#/dashboard?job_id=2020060802473235711835&role=local&party_id=0', 'job_dsl_path': '/data/projects/fate/python/jobs/2020060802473235711835/job_dsl.json', 'job_runtime_conf_path': '/data/projects/fate/python/jobs/2020060802473235711835/job_runtime_conf.json', 'logs_directory': '/data/projects/fate/python/logs/2020060802473235711835', 'namespace': 'fate_flow_test_breast', 'table_name': 'breast'}, 'jobId': '2020060802473235711835', 'retcode': 0, 'retmsg': 'success'}
running
running
running
running
running
running
running
running
running
running
running
running
success
Success


In [40]:
pipline = '''
{
    "components" : {
        "dataio_0": {
            "module": "DataIO",
            "input": {
                "data": {
                    "data": [
                        "args.train_data"
                    ]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["dataio"]
            },
                        "need_deploy": true
         },
        "hetero_feature_binning_0": {
            "module": "HeteroFeatureBinning",
            "input": {
                "data": {
                    "data": [
                        "dataio_0.train"
                    ]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["hetero_feature_binning"]
            }
        },
        "hetero_feature_selection_0": {
            "module": "HeteroFeatureSelection",
            "input": {
                "data": {"data": [
                        "hetero_feature_binning_0.train"
                    ]
                },
                "isometric_model": [
                    "hetero_feature_binning_0.hetero_feature_binning"
                ]
            },
            "output": {
                "data": ["train"],
                "model": ["selected"]
            }
        },
        "hetero_lr_0": {
            "module": "HeteroLR",
            "input": {
                "data": {
                    "train_data": ["hetero_feature_selection_0.train"]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["hetero_lr"]
            }
        },
        "evaluation_0": {
            "module": "Evaluation",
            "input": {
                "data": {
                    "data": ["hetero_lr_0.train"]
                }
            },
            "output": {
                "data": ["evaluate"]
            }
        }
    }
}
'''

config = '''
{
    "initiator": {
        "role": "guest",
        "party_id": 10000
    },
    "job_parameters": {
        "work_mode": 1
    },
    "role": {
        "guest": [10000],
        "host": [9999],
        "arbiter": [9999]
    },
    "role_parameters": {
        "guest": {
            "args": {
                "data": {
                    "train_data": [{"name": "breast_b", "namespace": "fate_flow_test_breast"}]
                }
            },
            "dataio_0":{
                "with_label": [true],
                "label_name": ["y"],
                "label_type": ["int"],
                "output_format": ["dense"]
            }
        },
        "host": {
            "args": {
                "data": {
                    "train_data": [{"name": "breast_a", "namespace": "fate_flow_test_breast"}]
                }
            },
             "dataio_0":{
                "with_label": [false],
                "output_format": ["dense"]
            }
        }
    },
    "algorithm_parameters": {
        "hetero_lr_0": {
            "penalty": "L2",
            "optimizer": "rmsprop",
            "eps": 1e-5,
            "alpha": 0.01,
            "max_iter": 3,
            "converge_func": "diff",
            "batch_size": 320,
            "learning_rate": 0.15,
            "init_param": {
                                "init_method": "random_uniform"
            }
        }
    }
}
'''

In [41]:
response = manager.submit_job(json.loads(pipline),json.loads(config))
manager.prettify(response, True)
stdout = json.loads(response.content)
jobId = stdout["jobId"]
query_condition = {
    "job_id":jobId
}

model_id, model_version = "", ""
for i in range(500):
    time.sleep(3)
    job_detail = manager.query_job(query_condition).json()
    final_status = job_detail["data"][0]["f_status"]
    print(final_status)
    
    if final_status == "failed":
        print("Failed")
        manager.prettify(job_detail, True)
        response = manager.fetch_job_log(jobId)
        raise Exception("Failed to run the job")
    if final_status == "success":
        print("Success")
        manager.prettify(response, True)
        output = json.loads(response.content)
        model_id, model_version = output["data"]["model_info"]["model_id"], output["data"]["model_info"]["model_version"]
        break

Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020060802475836992436&role=guest&party_id=10000",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020060802475836992436/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020060802475836992436/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020060802475836992436",
        "model_info": {
            "model_id": "arbiter-9999#guest-10000#host-9999#model",
            "model_version": "2020060802475836992436"
        }
    },
    "jobId": "2020060802475836992436",
    "retcode": 0,
    "retmsg": "success"
}
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
r

In [91]:
pipline = '''
{
    "components" : {
        "dataio_0": {
            "module": "DataIO",
            "input": {
                "data": {
                    "data": [
                        "args.train_data"
                    ]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["dataio"]
            },
                        "need_deploy": true
         },
        "hetero_feature_binning_0": {
            "module": "HeteroFeatureBinning",
            "input": {
                "data": {
                    "data": [
                        "dataio_0.train"
                    ]
                }
            },
            "output": {
                "data": ["train"],
                "model": ["hetero_feature_binning"]
            }
        },
        "hetero_feature_selection_0": {
            "module": "HeteroFeatureSelection",
            "input": {
                "data": {"data": [
                        "hetero_feature_binning_0.train"
                    ]
                },
                "isometric_model": [
                    "hetero_feature_binning_0.hetero_feature_binning"
                ]
            },
            "output": {
                "data": ["eval"],
                "model": ["selected"]
            }
        },
        "evaluation_0": {
            "module": "Evaluation",
            "input": {
                "data": {
                    "data": ["hetero_feature_selection_0.eval"]
                }
            },
            "output": {
                "data": ["evaluate"]
            }
        }
    }
}
'''

config = '''
{
    "initiator": {
        "role": "guest",
        "party_id": 10000
    },
    "job_parameters": {
        "work_mode": 1,
        "job_type": "predict",
        "model_id": "arbiter-9999#guest-10000#host-9999#model",
        "model_version": "2020060802475836992436"
    },
    "role": {
        "guest": [10000],
        "host": [9999],
        "arbiter": [9999]
    },
    "role_parameters": {
        "guest": {
            "args": {
                "data": {
                    "eval_data": [{"name": "breast_b", "namespace": "fate_flow_test_breast"}]
                }
            }
        },
        "host": {
            "args": {
                "data": {
                    "eval_data": [{"name": "breast_a", "namespace": "fate_flow_test_breast"}]
                }
            }
        }
    }
}
'''

In [92]:
response = manager.submit_job(json.loads(pipline), json.loads(config))
manager.prettify(response, True)
stdout = json.loads(response.content)
jobId = stdout["jobId"]
query_condition = {
    "job_id": jobId
}

Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020060806580950942753&role=guest&party_id=10000",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020060806580950942753/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020060806580950942753/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020060806580950942753",
        "model_info": {
            "model_id": "arbiter-9999#guest-10000#host-9999#model",
            "model_version": "2020060802475836992436"
        }
    },
    "jobId": "2020060806580950942753",
    "retcode": 0,
    "retmsg": "success"
}


In [ ]:
print(query_condition)

In [100]:
model_id, model_version = "", ""
for i in range(500):
    time.sleep(3)
    job_detail = manager.query_job(query_condition).json()
    final_status = job_detail["data"][0]["f_status"]
    print(final_status)
    
    if final_status == "failed":
        print("Failed")
        manager.prettify(job_detail, True)
        response = manager.fetch_job_log(jobId)
        raise Exception("Failed to run the job")
    if final_status == "success":
        print("Success")
        manager.prettify(response, True)
        output = json.loads(response.content)
        model_id, model_version = output["data"]["model_info"]["model_id"], output["data"]["model_info"]["model_version"]
        break

success
Success
Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020060806580950942753&role=guest&party_id=10000",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020060806580950942753/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020060806580950942753/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020060806580950942753",
        "model_info": {
            "model_id": "arbiter-9999#guest-10000#host-9999#model",
            "model_version": "2020060802475836992436"
        }
    },
    "jobId": "2020060806580950942753",
    "retcode": 0,
    "retmsg": "success"
}


In [105]:
manager.track_component_output_data("2020060806580950942753", "guest", "10000", "hetero_lr_0")

Success!
{
    "data": [
        [
            "0",
            0,
            0,
            0.0038378345762823474,
            {
                "0": 0.9961621654237176,
                "1": 0.0038378345762823474
            },
            "predict"
        ],
        [
            "1",
            0,
            0,
            0.4608240277994044,
            {
                "0": 0.5391759722005955,
                "1": 0.4608240277994044
            },
            "predict"
        ],
        [
            "2",
            0,
            0,
            0.025629183140501995,
            {
                "0": 0.974370816859498,
                "1": 0.025629183140501995
            },
            "predict"
        ],
        [
            "3",
            0,
            0,
            0.05433442394518767,
            {
                "0": 0.9456655760548123,
                "1": 0.05433442394518767
            },
            "predict"
        ],
        [
            "4",
         

<Response [200]>